In [86]:
from bs4 import BeautifulSoup
import html2text
import pandas as pd
from tqdm import tqdm 
tqdm.pandas()
from IPython.display import display, HTML
from src.preprocessing.news_parser import body_formatter
import re
pd.set_option(
    'display.max_colwidth', 2000
)

In [125]:
h = html2text.HTML2Text()
h.ignore_links = True
h.ignore_images = True
h.ignore_emphasis = True
h.dash_unordered_list = True
h.body_width = 0
h.drop_white_space = True

def body_formatter2(body):
    # Remove any segments at the end of a press release about the company
    # TODO: Maybe also tag news where there exists such a postable,
    # May be more reliable news than others ? 
    body = re.sub(f"(<b>About|for more information)((.|\n)*)+", "", body, re.IGNORECASE)
    
    soup = BeautifulSoup(body, features='html.parser')
    for t in soup.find_all('pre'):
        t.extract()
    for t in soup.find_all('table'):
        t.extract()

    new_body = str(soup)
    
    
    return h.handle(new_body)
    #return new_body


In [83]:
# Das ist nicht raw raw html, sondern nur raw...
df = pd.read_parquet("data/raw/story_df_raw_2010.parquet")

In [116]:
df.html_body.loc[74724]

'<div id="businesswire-body">STUDIO CITY, Calif.--(BUSINESS WIRE)--\n    <p>\n      Tix Corporation (Nasdaq: <a class="ticker" href="/stock/tixc#Nasdaq">TIXC</a>) today announced that the United States \n      Patent and Trademark Office has awarded and issued the company a United \n      States patent on a “Ticket Distribution System.”\n    </p>\n    <p>\n      The new patent covers a number of the key aspects of discount ticketing \n      brokerage operations and technological systems related to the sale and \n      distribution of unused tickets.\n    </p>\n    <p>\n      “We are thrilled about the issuance of this patent, which is the \n      culmination of eight years of effort," said Mitch Francis, chairman and \n      CEO, Tix Corporation. “This important corporate milestone reflects our \n      role as a leader and a pioneer in the business of brokering discount \n      tickets.”\n    </p>\n    <p>\n      <b>About TIX Corporation</b>\n    </p>\n    <p>\n      Tix Corporation (N

In [122]:
body_formatter2(df.html_body.loc[74724])

'STUDIO CITY, Calif.--(BUSINESS WIRE)-- \n\nTix Corporation (Nasdaq: TIXC) today announced that the United States Patent and Trademark Office has awarded and issued the company a United States patent on a “Ticket Distribution System.” \n\nThe new patent covers a number of the key aspects of discount ticketing brokerage operations and technological systems related to the sale and distribution of unused tickets. \n\n“We are thrilled about the issuance of this patent, which is the culmination of eight years of effort," said Mitch Francis, chairman and CEO, Tix Corporation. “This important corporate milestone reflects our role as a leader and a pioneer in the business of brokering discount tickets.” \n'

In [84]:
current_bodies = df.html_body.progress_map(body_formatter)

  0%|          | 0/2500 [00:00<?, ?it/s]

100%|██████████| 2500/2500 [00:14<00:00, 170.16it/s]


In [123]:
new_bodies = df.html_body.progress_map(body_formatter2)

100%|██████████| 2500/2500 [00:10<00:00, 237.54it/s]


In [130]:
pd.concat([current_bodies, new_bodies], axis=1).iloc[150:200, :]

,html_body,html_body
74819,"OAKLAND, Calif.--(BUSINESS WIRE)-- \n\nZhone Technologies, Inc. (NASDAQ: ZHNE), a global leader in network access solutions, announced today that Brian Caskey has joined the company as Chief Marketing Officer effective immediately. Caskey brings a 25 year track record in global marketing, product strategy, business development and global branding to Zhone where he will oversee all aspects of worldwide marketing. \n\nPrior to joining Zhone, Caskey served as Vice President of Worldwide Marketing for UTStarcom where he was instrumental in growing the IPTV and NGN business. There, he spearheaded a company-wide global rebranding campaign, launched multiple new product lines and led strategic sales initiatives that resulted in top-line revenue growth while increasing gross margins. Caskey’s accomplishments included critical innovations in IPTV marketing and awareness that resulted in industry leading deployments in both India and China. \n\nPrior to UTStarcom, Caskey worked for 3Com Corporation where he was responsible for strategic technical marketing and advanced service provider solutions. At 3Com, he oversaw technical solutions marketing and architecture of Next Generation networks for some of the industry’s leading Tier 1 service providers. \n\n“Brian is results-driven with a global mindset and the right mix of strategic vision and practical expertise,” said Mory Ejabat, Chairman and CEO for Zhone. “He has deep understanding of both the challenges and the opportunities facing carriers today, with a strong record in guiding teams to profitability.” \n\n“Zhone made solid gains in product strategy with the introduction of its MXK platform and that provides a motivating opportunity for growth and innovation ahead,” said Brian Caskey, Zhone’s Chief Marketing Officer. “I’m looking forward to making a positive contribution to advancing Zhone's global brand.” \n\nPrior to 3Com, Caskey held marketing and management positions at Timeplex and GTE Spacenet. He earned his deg...","\n\nOAKLAND, Calif.--(BUSINESS WIRE)-- \n\nZhone Technologies, Inc. (NASDAQ: ZHNE), a global leader in network access solutions, announced today that Brian Caskey has joined the company as Chief Marketing Officer effective immediately. Caskey brings a 25 year track record in global marketing, product strategy, business development and global branding to Zhone where he will oversee all aspects of worldwide marketing. \n\nPrior to joining Zhone, Caskey served as Vice President of Worldwide Marketing for UTStarcom where he was instrumental in growing the IPTV and NGN business. There, he spearheaded a company-wide global rebranding campaign, launched multiple new product lines and led strategic sales initiatives that resulted in top-line revenue growth while increasing gross margins. Caskey’s accomplishments included critical innovations in IPTV marketing and awareness that resulted in industry leading deployments in both India and China. \n\nPrior to UTStarcom, Caskey worked for 3Com Corporation where he was responsible for strategic technical marketing and advanced service provider solutions. At 3Com, he oversaw technical solutions marketing and architecture of Next Generation networks for some of the industry’s leading Tier 1 service providers. \n\n“Brian is results-driven with a global mindset and the right mix of strategic vision and practical expertise,” said Mory Ejabat, Chairman and CEO for Zhone. “He has deep understanding of both the challenges and the opportunities facing carriers today, with a strong record in guiding teams to profitability.” \n\n“Zhone made solid gains in product strategy with the introduction of its MXK platform and that provides a motivating opportunity for growth and innovation ahead,” said Brian Caskey, Zhone’s Chief Marketing Officer. “I’m looking forward to making a positive contribution to advancing Zhone's global brand.” \n\nPrior to 3Com, Caskey held marketing and management positions at Timeplex and G

In [96]:
idx = 0

In [89]:
df.html_body.iloc[idx]

'<div id="prnewswire-body">\n\n<p/><p/><p>LOS ANGELES, Jan. 3 /PRNewswire-FirstCall/ -- <a href="http://www.rentrak.com/">Rentrak Corporation</a> (Nasdaq: <a class="ticker" href="/stock/rent#Nasdaq">RENT</a>), today announced the weekend theatrical box office numbers for the weekend period of January 1 through January 3, 2010 according to the company\'s <a href="http://www.rentrak.com/section/media/theatrical/box_office.html">Box Office Essentials(TM)</a> theatrical box office data collection and analytical service.</p>\n<p/><p>The twelve estimated top producing motion pictures for the weekend, per data collected as of Sunday, January 3, 2010 include:</p>\n<p/><pre>\n\n\n    Rank  Title         Distributor      Weekend Gross Cume         Locations\n    ----  -----         -----------      ------------- ----         ---------\n     1 Avatar           20th Century Fox    68,300,000  352,111,074      3461\n    -- ------           ----------------    ----------  -----------      ----\n    

In [ ]:
html_body = df.html_body.iloc[idx]
display(HTML(html_body))

In [51]:
bodies.iloc[idx]

"LOS ANGELES, Jan. 3 /PRNewswire-FirstCall/ -- Rentrak Corporation (Nasdaq: RENT), today announced the weekend theatrical box office numbers for the weekend period of January 1 through January 3, 2010 according to the company's Box Office Essentials(TM) theatrical box office data collection and analytical service.\n\nThe twelve estimated top producing motion pictures for the weekend, per data collected as of Sunday, January 3, 2010 include:\n    \n    \n    \n    \n        Rank  Title         Distributor      Weekend Gross Cume         Locations\n        ----  -----         -----------      ------------- ----         ---------\n         1 Avatar           20th Century Fox    68,300,000  352,111,074      3461\n        -- ------           ----------------    ----------  -----------      ----\n         2 Sherlock Holmes  Warner Bros.        38,385,000  140,675,000      3626\n        -- ---------------  -----------         ----------  -----------      ----\n         3 Alvin And The\n      

In [60]:
new_bodies.iloc[idx]

"LOS ANGELES, Jan. 3 /PRNewswire-FirstCall/ -- Rentrak Corporation (Nasdaq: RENT), today announced the weekend theatrical box office numbers for the weekend period of January 1 through January 3, 2010 according to the company's Box Office Essentials(TM) theatrical box office data collection and analytical service.\n\nThe twelve estimated top producing motion pictures for the weekend, per data collected as of Sunday, January 3, 2010 include:\n    \n    \n    \n    \n        Rank  Title         Distributor      Weekend Gross Cume         Locations\n        ----  -----         -----------      ------------- ----         ---------\n         1 Avatar           20th Century Fox    68,300,000  352,111,074      3461\n        -- ------           ----------------    ----------  -----------      ----\n         2 Sherlock Holmes  Warner Bros.        38,385,000  140,675,000      3626\n        -- ---------------  -----------         ----------  -----------      ----\n         3 Alvin And The\n      